In [51]:
import numpy as np
import os
from PIL import Image
import csv

In [87]:
train_img_path = './train_images'
test_img_path = './test_images'
train_label_file = 'train.csv'
test_label_file = 'test.csv'

In [92]:
train_img = []
train_label = []

In [93]:
for file in os.listdir(train_img_path):
    img = Image.open(train_img_path + '/' + file)
    img = img.convert('RGB')
    train_img.append(np.array(img))
train_img = np.array(train_img)
train_img.shape

(2528, 512, 512, 3)

In [94]:
i = 0
with open(train_label_file, newline = '') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        if i != 0:
            train_label.append(int(row[1]))
        i = i + 1
train_label = np.array(train_label)
train_label.shape

(2528,)